In [1]:
from data_generators import get_iterator, get_dataset
from classifiers import theme_classifier
import torch
import torch.nn as nn
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [2]:
train_dataset, val_dataset, test_dataset, review_text_FIELD, theme_FIELD = get_dataset(vectors = GLOVE_EMBEDDING)

In [3]:
batch_size = 20
train_iter = get_iterator(train_dataset, batch_size, train=True, shuffle=True, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=True, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=True, repeat=False)

In [46]:
def transtaltion2string(raw_translations, target_vocab = review_text_FIELD, max_words=30000):
    string_translations = []
    for raw_sentence in raw_translations:
        string_sentence = []
        for i, word_idx in enumerate(raw_sentence):
            if i == max_words: break
            word = target_vocab.vocab.itos[word_idx]
            if word == '<eos>':
                break
            if word != '<sos>':
                string_sentence.append(word)
        string_translations.append(string_sentence)

    return string_translations

In [13]:
from collections import Counter

In [96]:
first_words = []
selected_labels = []
lens = []
if True:
    for i, batch in enumerate(test_iter):
        labels = batch.theme.cuda().long() - 1
        batch = batch.review_text.cuda().long()
        if batch.shape[0] > 3:
            selected_labels += labels.tolist()
            first_words += batch[1,:].tolist()
            lens.append(batch.shape[0]-1)

In [100]:
avg_len = np.mean(lens)
avg_len

22.592261904761905

In [64]:
print(theme_FIELD.vocab.itos[1:])
print(dict(Counter(selected_labels)))

['other', 'plot', 'acting', 'effect', 'production']
{0: 37614, 3: 2604, 1: 38078, 2: 6593, 4: 2454}


In [86]:
# remove duplicates
selected_first_words = list(set(first_words))
print(len(first_words), len(selected_first_words))

# clean for first words that are keywords
first_words_text = transtaltion2string([list(selected_first_words)])[0]
first_words_text = list(map(lambda x: theme_classifier(x) == 'other', first_words_text))
selected_first_words = np.array(selected_first_words)[first_words_text].tolist()
print(len(selected_first_words))

87343 4519
4481


In [87]:
#test_list = list(test_iter)

In [88]:
from baseline_model import BaseModel, repackage_hidden

In [89]:
with open('./baseline/best_model_base_model_ppl_67.5461793533482.model', 'rb') as file:
    model = torch.load(file)
    model.eval()

In [90]:
vocab_size = 12304

In [91]:
import numpy as np

In [92]:
BOS_WORD = '<sos>'
EOS_WORD = '<eos>'

In [93]:
def generate_next_n_words(self, first_word, label, max_len = 3, # ngram=3
                            target_vocab = review_text_FIELD, top_k = 10):
    self.eval()
    with torch.no_grad():      

        labels = torch.tensor([label]).cuda().long()
        source = torch.tensor([first_word]).unsqueeze(1).cuda().long()
        
        eos_idx = target_vocab.vocab.stoi[EOS_WORD]
        y_hat = source[:1,:]
        y_hat = y_hat.unsqueeze(-1)

        prediction_idxs = [[y_hat[0]]]
        prediction_probs = [0]

        next_hidden = [None]
        
        for t in range(max_len):
            current_idxs = []
            current_probs = []
            current_hidd = [] 

            for k in range(y_hat.shape[-1]):
                y_hat_k = y_hat[:,:,k]
                current_hidd_k = next_hidden[k]
                output, hidden = self(y_hat_k, labels, current_hidd_k)
                current_hidd += [hidden]*top_k
                top_probs, top_idx = output.topk(top_k, -1)
                current_idxs.append(top_idx.squeeze(0).squeeze(0))
                current_probs.append(top_probs.squeeze(0).squeeze(0))
            current_idxs = torch.cat(current_idxs)
            current_probs = torch.cat(current_probs)

            next_args = torch.sort(current_probs, -1, True)[1][:top_k] 

            next_idxs, next_probs, next_hidden = [], [], []
            tmp_pred_idxs, tmp_pred_probs = [], []
            for idx in next_args:
                chain_head = idx//top_k

                next_idx = current_idxs[idx].int()
                next_prob = current_probs[idx]
                sentence_so_far = prediction_idxs[chain_head]
                
                if len(sentence_so_far)>0 and sentence_so_far[-1] == eos_idx:
                    tmp_pred_idxs.append(sentence_so_far + [eos_idx])
                    tmp_pred_probs.append(prediction_probs[chain_head])
                else:
                    tmp_pred_idxs.append(sentence_so_far + [next_idx])
                    tmp_pred_probs.append(prediction_probs[chain_head] + next_prob)

                next_idxs.append(next_idx.unsqueeze(0))
                next_hidden.append(current_hidd[idx])

            prediction_idxs = tmp_pred_idxs
            prediction_probs = tmp_pred_probs
            next_idxs = torch.cat(next_idxs)

            y_hat = next_idxs.unsqueeze(0).unsqueeze(0).long()
            
        prediction_probs = torch.tensor(prediction_probs)
        prediction_idxs = torch.tensor(prediction_idxs)
        correct_order = torch.argsort(prediction_probs, descending=True).long()
        
        prediction_idxs = torch.index_select(prediction_idxs, 0, correct_order)

        return prediction_idxs, labels

In [94]:
import pandas as pd

In [106]:
#review_text_FIELD.vocab.itos
theme_FIELD.vocab.itos

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']

In [107]:
avg_len

22.592261904761905

In [132]:
results = []
for lid, label in enumerate(theme_FIELD.vocab.itos[1:]):
    print(lid, label)
    for wid, word0 in enumerate(selected_first_words):
        if wid % 50 == 0 : print('{}/{}'.format(wid, len(selected_first_words)))
        pred, label = generate_next_n_words(model, word0, lid, max_len=20, top_k=1) #int(avg_len)
        pred = transtaltion2string(pred[:1])
        string = ' '.join(pred[0])
        predicted_class = theme_classifier(string)
        results.append([theme_FIELD.vocab.itos[label+1], string, predicted_class])

0 other
0/4481
50/4481
100/4481
150/4481
200/4481
250/4481
300/4481
350/4481
400/4481
450/4481
500/4481
550/4481
600/4481
650/4481
700/4481
750/4481
800/4481
850/4481
900/4481
950/4481
1000/4481
1050/4481
1100/4481
1150/4481
1200/4481
1250/4481
1300/4481
1350/4481
1400/4481
1450/4481
1500/4481
1550/4481
1600/4481
1650/4481
1700/4481
1750/4481
1800/4481
1850/4481
1900/4481
1950/4481
2000/4481
2050/4481
2100/4481
2150/4481
2200/4481
2250/4481
2300/4481
2350/4481
2400/4481
2450/4481
2500/4481
2550/4481
2600/4481
2650/4481
2700/4481
2750/4481
2800/4481
2850/4481
2900/4481
2950/4481
3000/4481
3050/4481
3100/4481
3150/4481
3200/4481
3250/4481
3300/4481
3350/4481
3400/4481
3450/4481
3500/4481
3550/4481
3600/4481
3650/4481
3700/4481
3750/4481
3800/4481
3850/4481
3900/4481
3950/4481
4000/4481
4050/4481
4100/4481
4150/4481
4200/4481
4250/4481
4300/4481
4350/4481
4400/4481
4450/4481
1 plot
0/4481
50/4481
100/4481
150/4481
200/4481
250/4481
300/4481
350/4481
400/4481
450/4481
500/4481
550/4481
600

In [133]:
DF = pd.DataFrame(results, columns=['true_label', 'generation', 'predicted'])

In [150]:
DF = DF[DF.generation.apply(lambda x: x.split()[0] != '<unk>')]

In [151]:
DF.to_csv('DF20.csv', index=False)

In [152]:
DF.true_label.unique()

array(['other', 'plot', 'acting', 'effect', 'production'], dtype=object)

In [153]:
print('baseline 1c - global acc score')
(DF.true_label == DF.predicted).mean()

baseline 1c - global acc score


0.7670089285714285

In [154]:
print('baseline 1c - acc score per class')
for case in DF.true_label.unique():
    print(case, (DF[DF.true_label==case].true_label == DF[DF.true_label==case].predicted).mean())

baseline 1c - acc score per class
other 0.9716517857142857
plot 0.5696428571428571
acting 0.8479910714285714
effect 0.5810267857142857
production 0.8647321428571428


In [155]:
from sklearn.metrics import confusion_matrix

In [156]:
cm = confusion_matrix(DF.true_label, DF.predicted, labels=DF.true_label.unique())

In [157]:
print('baseline 1c - confusion matrix')
print(DF.true_label.unique())
(cm / cm.sum(-1)).round(2)

baseline 1c - confusion matrix
['other' 'plot' 'acting' 'effect' 'production']


array([[0.97, 0.02, 0.  , 0.01, 0.  ],
       [0.43, 0.57, 0.  , 0.  , 0.  ],
       [0.14, 0.01, 0.85, 0.  , 0.  ],
       [0.41, 0.01, 0.  , 0.58, 0.  ],
       [0.13, 0.  , 0.  , 0.  , 0.86]])